# ATLAS Multihop QA Benchmarking

In [ ]:
# print python version
print("Python version:", __import__('sys').version)

In [ ]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '1'  # Set to the GPU you want to use, or '0' for the first GPU
import torch
num_gpus = torch.cuda.device_count()
print("number of GPUs available:", torch.cuda.device_count())
for i in range(num_gpus):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

In [2]:
from atlas_rag.vectorstore.embedding_model import SentenceEmbedding
from sentence_transformers import SentenceTransformer
# Load the SentenceTransformer model
encoder_model_name = 'all-MiniLM-L6-v2'
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')
sentence_encoder = SentenceEmbedding(sentence_model)

In [3]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from atlas_rag.llm_generator import LLMGenerator
from configparser import ConfigParser
config = ConfigParser()
config.read('config.ini')
# Added support for Azure Foundry. To use it, please do az-login in cmd first.
reader_model_name = "deepseek-ai/DeepSeek-V3-0324"
reader_model_name = "DeepSeek-V3-0324"
connection = AIProjectClient(
    endpoint=config["urls"]["AZURE_URL"],
    credential=DefaultAzureCredential(),
)
client = connection.inference.get_azure_openai_client(api_version="2024-12-01-preview")
llm_generator = LLMGenerator(client=client, model_name=reader_model_name)

In [ ]:
from openai import OpenAI
from atlas_rag.llm_generator import LLMGenerator
from configparser import ConfigParser
# Load OpenRouter API key from config file
config = ConfigParser()
config.read('config.ini')
reader_model_name = "meta-llama/Llama-3.3-70B-Instruct"
client = OpenAI(
  base_url="https://api.deepinfra.com/v1/openai",
  api_key=config['settings']['DEEPINFRA_API_KEY'],
)
llm_generator = LLMGenerator(client=client, model_name=reader_model_name)

In [ ]:
from atlas_rag.vectorstore import create_embeddings_and_index
keyword = 'musique'
working_directory = f'/data/httsangaj/atomic-rag/8b'
data = create_embeddings_and_index(
    sentence_encoder=sentence_encoder,
    model_name = encoder_model_name,
    working_directory=working_directory,
    keyword=keyword,
    include_concept=True,
    include_events=True,
    normalize_embeddings= True,
    text_batch_size=64,
    node_and_edge_batch_size=64,
)

In [5]:
from atlas_rag.evaluation import BenchMarkConfig
benchmark_config = BenchMarkConfig(
    dataset_name= 'musique',
    question_file= "benchmark_data/musique.json",
    include_concept=True,
    include_events=True,
    reader_model_name=reader_model_name,
    encoder_model_name=encoder_model_name,
    number_of_samples=2, # -1 for all samples
)

In [6]:
from atlas_rag import setup_logger
logger = setup_logger(benchmark_config)

In [ ]:
# Initialize desired RAG method for benchmarking
from atlas_rag.retriever import HippoRAG2Retriever
hipporag2_retriever = HippoRAG2Retriever(
    llm_generator=llm_generator,
    sentence_encoder=sentence_encoder,
    data = data,
    logger=logger
)

In [ ]:
# start benchmarking
from atlas_rag.evaluation import RAGBenchmark
benchmark = RAGBenchmark(config=benchmark_config, logger=logger)
benchmark.run([hipporag2_retriever], llm_generator=llm_generator) 
# benchmark.run([hipporag2_retriever], llm_generator=llm_generator, use_react=True) # use_react=True to enable RAG with REACT